# More Data Preprocessing (with Pandas)

Pandas is a very useful data analytics package within Python.

Let's start with some random data.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame(np.random.randn(5, 3), index=['a', 'c', 'e', 'f', 'h'], columns=['one', 'two', 'three'])
df

,one,two,three
a,1.173730,-1.012183,-0.408465
c,-0.122136,-0.258496,1.117583
e,-1.231365,1.002752,-0.533629
f,2.561557,0.016030,2.337315
h,0.080111,-2.122272,0.168148


We know with Pandas we can mix & match data types, so let us add two more columns, named 'four' and 'five':

In [3]:
df['four'] = 'bar'
df['five'] = df['one'] > 0
df

,one,two,three,four,five
a,1.173730,-1.012183,-0.408465,bar,True
c,-0.122136,-0.258496,1.117583,bar,False
e,-1.231365,1.002752,-0.533629,bar,False
f,2.561557,0.016030,2.337315,bar,True
h,0.080111,-2.122272,0.168148,bar,True


Adding new rows is also simple. Below we include three extra empty rows:

In [4]:
df2 = df.reindex(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'])
df2

,one,two,three,four,five
a,1.173730,-1.012183,-0.408465,bar,True
b,NaN,NaN,NaN,NaN,NaN
c,-0.122136,-0.258496,1.117583,bar,False
d,NaN,NaN,NaN,NaN,NaN
e,-1.231365,1.002752,-0.533629,bar,False
f,2.561557,0.016030,2.337315,bar,True
g,NaN,NaN,NaN,NaN,NaN
h,0.080111,-2.122272,0.168148,bar,True


# Your Turn Here

Do you still remember how to index a row/column?

In [8]:
#### index row 'c' below
df2.loc['c']

one     -0.122136
two     -0.258496
three     1.11758
four          bar
five        False
Name: c, dtype: object

In [9]:
#### index column 'two' below
df2.two

a   -1.012183
b         NaN
c   -0.258496
d         NaN
e    1.002752
f    0.016030
g         NaN
h   -2.122272
Name: two, dtype: float64

pandas has two functions isnull() and notnull() that return boolean objects when called.

In [10]:
pd.isnull(df2['one'])

a    False
b     True
c    False
d     True
e    False
f    False
g     True
h    False
Name: one, dtype: bool

In [11]:
pd.notnull(df2['one'])

a     True
b    False
c     True
d    False
e     True
f     True
g    False
h     True
Name: one, dtype: bool

Missing values propagate naturally through arithmetic operations between pandas objects.

In [12]:
a = df[['one','two']]
a['one']['a':'e'] = float('nan')
a

/opt/tljh/user/lib/python3.6/site-packages/pandas/core/series.py:935: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_with(key, value)
/opt/tljh/user/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,one,two
a,NaN,-1.012183
c,NaN,-0.258496
e,NaN,1.002752
f,2.561557,0.016030
h,0.080111,-2.122272


In [13]:
b = df[['one','two','three']]
b

,one,two,three
a,1.173730,-1.012183,-0.408465
c,-0.122136,-0.258496,1.117583
e,-1.231365,1.002752,-0.533629
f,2.561557,0.016030,2.337315
h,0.080111,-2.122272,0.168148


In [15]:
b + a

,one,three,two
a,NaN,NaN,-2.024366
c,NaN,NaN,-0.516992
e,NaN,NaN,2.005505
f,5.123114,NaN,0.032061
h,0.160223,NaN,-4.244545


# How to deal with Missing Values

## Deleting Missing Values

The simplest method is always dropping all missing values - but it is highly **discouraged!!!**

In [16]:
a['one'].dropna()

f    2.561557
h    0.080111
Name: one, dtype: float64

By default, dropna() will drop any row containing **NaN** values, but you can change that by using the *axis=* and *thresh=* arguments.

**NOTE**: Dropping rows or columns have different uses.

In [17]:
#### This statement drop any column with NaN values
a.dropna(axis=1)

,two
a,-1.012183
c,-0.258496
e,1.002752
f,0.016030
h,-2.122272


In [18]:
#### thresh determines how many non-NaN values a column/row should have without being dropped
c = a + b
c.dropna(axis=1, thresh=2)

,one,two
a,NaN,-2.024366
c,NaN,-0.516992
e,NaN,2.005505
f,5.123114,0.032061
h,0.160223,-4.244545


You can also use the *how=* argument to determine how do you want to remove the NaN values.

In [19]:
#### By default, dropna() drops column/row with any NaN values
#### how = 'all' changes that to dropping column/row with all NaN values
c.dropna(axis=1, how='all')

,one,two
a,NaN,-2.024366
c,NaN,-0.516992
e,NaN,2.005505
f,5.123114,0.032061
h,0.160223,-4.244545


You can refer to the [pandas.dropna() docs](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html) for more information.

## Imputing Missing Values

Imputing means filling missing values - you can do that when the missing and non-missing values are in some type of relationship.

In [20]:
my_series = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
my_series

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [21]:
#### you can fill missing values with a specific value (0)
my_series.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

Alternatively, we can specify a forward-fill to propagate the previous value forward:

In [22]:
# forward-fill
my_series.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Or we can specify a back-fill to propagate the next values backward:

In [23]:
my_series.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

## Your Turn Here
Aforementioned method can be applied to dataframes. 

In [24]:
#### Let us generate a random dataframe
rand_df = pd.DataFrame(np.random.randn(5, 3), 
                  index=['a', 'b', 'c', 'd', 'e'],
                  columns=['one', 'two', 'three'])
rand_df = rand_df.mask(np.random.random(rand_df.shape) < .3)
rand_df

,one,two,three
a,0.668504,-0.197383,NaN
b,-0.941630,0.710686,NaN
c,0.039165,-1.038984,0.144422
d,-1.987875,-0.627242,0.682455
e,-0.681808,1.160510,0.074817


Your tasks are as follows:

- fill missing values in column **one** with value 1;
- fill missing values in column **two** with forward-filling;
- fill missing values in column **three** with backward-filling.

In [28]:
#### insert your code here
rand_df.one.fillna(1,inplace=True)
rand_df.two.fillna(method='ffill',inplace=True)
rand_df.three.fillna(method='bfill',inplace=True)
rand_df

,one,two,three
a,0.668504,-0.197383,0.144422
b,-0.941630,0.710686,0.144422
c,0.039165,-1.038984,0.144422
d,-1.987875,-0.627242,0.682455
e,-0.681808,1.160510,0.074817


A useful approach for imputing your missing data is to use mean/mode to replace missing data - the reason behind this logic is that if we are going to *guess* the values of the missing data, the highest chance would be guessing it to be the mean/mode if the data follows **normal** distribution.

In [29]:
#### let us generate another DF
my_df = pd.DataFrame(np.random.randn(5, 3), 
                  index=['a', 'b', 'c', 'd', 'e'],
                  columns=['A', 'B', 'C'])
my_df = my_df.mask(np.random.random(my_df.shape) < .3)
my_df

,A,B,C
a,NaN,2.032375,0.397380
b,1.100382,-0.754305,NaN
c,0.637154,NaN,-1.597821
d,NaN,NaN,NaN
e,0.769073,0.968315,NaN


In [30]:
#### Let us check if there is any missing value in the df
my_df.isnull().values.any()

True

In [31]:
#### Then we are getting the mean of the DF
#### Note that since we only 
my_df.mean()

A    0.835536
B    0.748795
C   -0.600221
dtype: float64

In [32]:
df_filled = my_df.fillna(my_df.mean())
df_filled

,A,B,C
a,0.835536,2.032375,0.397380
b,1.100382,-0.754305,-0.600221
c,0.637154,0.748795,-1.597821
d,0.835536,0.748795,-0.600221
e,0.769073,0.968315,-0.600221


In [33]:
#### Now let us check again if there is any missing values
df_filled.isnull().values.any()

False

More info regarding how to handle missing data can be found [here](https://machinelearningmastery.com/handle-missing-data-python/).

# Other Tasks in Data Preprocessing

- Handling categorical data (coding)
- Handling imbalanced data
- feature engineering

These topics will be covered in later part of this class.